In [1]:
%matplotlib widget
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import ticker, cm
from matplotlib.ticker import LogFormatter 
from Utils import *
from scipy.optimize import curve_fit
import copy
import os
import h5py
import json

import PyPDF2                       # Per unire i PDF alla fine
import glob                         # Per manipolazione dei path

from collections.abc import Iterable


In [2]:
config_file = r"./config.json"
print (config_file)

with open(config_file, "r") as f:
            dizi = json.load(f)
        
dizi
Swan = False

./config.json


In [ ]:
fig, axes = plt.subplots(3,3)
fig.set_size_inches(8,8)
titles = ['George','John','Paul']
time_peak_Beatles =[]
for i in range(3):
    ax = axes[i,0]
    h, bins = np.histogram(ph[:,2+(i*2)]+ph[:,3+(i*2)], bins = 200, )
    binc = bins[:-1] + (bins[1] - bins[0]) / 2
    ax.set_title(titles[i]+' Ph')
    ax.set_yscale('log')
    ax.plot(binc, h)
    ax.plot(binc[np.argmax(h)],np.max(h),'*')
for i in range(3):
    ax = axes[i,1]
    h, bins = np.histogram(tm[:,2+(i*2)], bins = 200, )
    binc = bins[:-1] + (bins[1] - bins[0]) / 2
    ax.set_title('Time '+titles[i]+'1')
    ax.set_yscale('log')
    ax.plot(binc, h)
    ax.plot(binc[np.argmax(h)],np.max(h),'*')
    print(binc[np.argmax(h)],np.max(h))
    time_peak_Beatles.append(binc[np.argmax(h)])
for i in range(3):
    ax = axes[i,2]
    h, bins = np.histogram(tm[:,3+(i*2)], bins = 200, )
    binc = bins[:-1] + (bins[1] - bins[0]) / 2
    ax.set_title('Time '+ titles[i]+'2')
    ax.set_yscale('log')
    ax.plot(binc, h)
    ax.plot(binc[np.argmax(h)],np.max(h),'*')
    print(binc[np.argmax(h)],np.max(h))
    time_peak_Beatles.append(binc[np.argmax(h)])

In [ ]:
print(Average(time_peak_Beatles))
fig, axes = plt.subplots(3,2)
fig.set_size_inches(8,8)
titles = ['George','John','Paul']
sipm_time = [Average(time_peak_Beatles) -10 ,Average(time_peak_Beatles) +10]
peak =[]

energy_loss_mip_in_PWO= 1.229*4.5*8.28 #MeV
# energy_loss_mip_in_LG= 1.255 #MeV
print(energy_loss_mip_in_PWO,'MeV')
time_logic_Beatles = [0,0,0]
for i in range(3):
    Si_tm_logic_1 = (tm[:,2+(i*2)] < sipm_time[1] ) & (tm[:,2+(i*2)] > sipm_time[0])
    Si_tm_logic_2 = (tm[:,3+(i*2)] < sipm_time[1]) & (tm[:,3+(i*2)] > sipm_time[0])
    time_logic_Beatles[i] = (Si_tm_logic_1) & (Si_tm_logic_2)

time_logic_Beatles= np.array(time_logic_Beatles).T
print(time_logic_Beatles.shape)
time_logic_si = np.any(time_logic_Beatles,axis=1)
print(time_logic_si.shape)

for i in range(3):
    ax = axes[i,0]
    h, bins = np.histogram(ph[:,2+(i*2)][time_logic_si]+ph[:,3+(i*2)][time_logic_si], bins = 200, )
    binc = bins[:-1] + (bins[1] - bins[0]) / 2
    ax.set_title(titles[i])
    ax.set_yscale('log')
    ax.plot(binc, h)
    ax.plot(binc[np.argmax(h)],np.max(h),'*')
    # ax.hist(ph[:,2+(i*2)][time_logic_si]+ph[:,3+(i*2)][time_logic_si],200)
    peak.append([binc[np.argmax(h)],np.max(h)])



print(peak)
print(peak[0][0],peak[1][0],peak[2][0])


#Equalize the SiPM around Jhon
Equalize = True
if Equalize:
    George = (ph[:,2]+ph[:,3])*(peak[1][0]/peak[0][0])
    Paul = (ph[:,6]+ph[:,7])*(peak[1][0]/peak[2][0])
    John = (ph[:,4]+ph[:,5])
    Beatles = [George,John,Paul]
    dizi['George_correction'] = (peak[1][0]/peak[0][0])
    dizi['Paul_correction'] = (peak[1][0]/peak[2][0])
    dizi['John_correction'] = (peak[1][0]/peak[1][0])

    with open(config_file, "w") as f:
            json.dump(dizi, f, indent=4)
    
for i in range(3):
    ax = axes[i,1]
    h, bins = np.histogram(Beatles[i], bins = 200, )
    binc = bins[:-1] + (bins[1] - bins[0]) / 2
    #ax.hist2d(ph[:,2+(i*2)]+ph[:,3+(i*2)],Beatles[i], bins=[200,200],cmap = my_cmap, norm=mpl.colors.LogNorm())
    ax.set_title(titles[i]+" Eq")
    ax.set_yscale('log')
    ax.plot(binc,h)





In [ ]:
logic_cherry_el = (ph[:, 0]> 200)
if fit_time: cut_tm = [time_popt[1]-time_popt[2], time_popt[1]+time_popt[2]]
if fit_mip: 
    cut_ph_mip = [mip_popt[1]-mip_popt[2], mip_popt[1]+mip_popt[2]]
    logic_mip = (ph[:,1]>[cut_ph_mip[0]]) & (ph[:,1]<[cut_ph_mip[1]])
    x_mip = xpos[logic_mip]
    ph_mip = ph[logic_mip]
    tm_mip= tm[logic_mip]
    xinfo_mip = xinfo[logic_mip]
if fit_el :
    cut_ph_el = [el_popt[1]-el_popt[2], el_popt[1]+el_popt[2]]
    logic_el = (logic_cherry_el) & (ph[:,1]>[cut_ph_el[0]]) & (ph[:,1]<[cut_ph_el[1]])
    #filtri cherenkov MIB vs EL

    x_el = xpos[logic_el]
    ph_el = ph[logic_el]
    tm_el = tm[logic_el]
    xinfo_el = xinfo[logic_el]

#positions of crystal horizzontal
fig, axes = plt.subplots(3,3)
fig.set_size_inches(10,6)
fig.subplots_adjust(wspace=.4,hspace=.8)

titles = ['George','John','Paul']
for i in range(3):
    ax = axes[0,i]
    ax.hist(Beatles[i], bins = 110)
    ax.set_xlabel('Ph')
    ax.set_xlim(0,30000)
    ax.set_yscale('log')
    ax.set_title(titles[i])
    
for i in range(3):
    ax = axes[1,i]
    # ax.hist(ph[:,2+(i*2)]+ph[:,3+(i*2)], bins = 1100)
    ax.hist(Beatles[i], bins = 500)
    ax.set_xlabel('Ph')
    ax.set_xlim(0,5000)
    ax.set_yscale('log')
    # ax.set_ylim(0,30)
    ax.set_title(titles[i])

my_cmap = copy.copy(mpl.cm.jet) # copy the default cmap
my_cmap.set_bad(my_cmap(0))

# print (xpos[:,2].shape)
# print (time_logic_si.shape)
# print (xpos[:,2][time_logic_si].shape)
# print (Beatles[i].shape)

for i in range(3):
    ax = axes[2,i]
    # ax.hist2d(xpos[:,2][time_logic_si][Beatles[i]<2000],Beatles[i][Beatles[i]<2000], bins=[110,500],cmap = my_cmap,norm=mpl.colors.LogNorm())
    ax.hist2d(xpos[:,2],Beatles[i], bins=[110,500],cmap = my_cmap,norm=mpl.colors.LogNorm())
    ax.set_title(titles[i])
    ax.set_xlim(-2,9)
    # ax.set_ylim(0,2000)
    ax.set_xlabel('xpos')
    ax.set_ylabel('ph')